<a href="https://colab.research.google.com/github/ttobaegi/ML_Python/blob/master/EDA_BankMarketing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 구글 Colab 서버 세팅

In [1]:
print('===== OS =====')
!cat /etc/issue.net

print('===== CPU =====')
!cat /proc/cpuinfo

print('===== Memory =====')
!cat /proc/meminfo

print('===== Disk =====')
!df -h

print('===== GPU =====')
!nvidia-smi

===== OS =====
Ubuntu 18.04.5 LTS
===== CPU =====
processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physic

In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')         # Suppress Depreciation and Incorrect Usage Warnings

## 나눔고딕 폰트 설치
!apt-get install fonts-nanum*
!apt-get install fontconfig
!fc-cache -fv
!cp /usr/share/fonts/truetype/nanum/Nanum* /usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data/fonts/ttf/
!rm -rf /content/.cache/matplotlib/*

# 한글 설정
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib as mpl
import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# 나눔고딕 폰트 설정 세팅
matplotlib.front_manager._rebuild()
matplotlib.rc('font', family = 'NanumGothic')
# 나눔고딕 폰트 그래프내 깨지는 경우 대비
mpl.rcParams['axes.unicode_minus'] = False

import seaborn as sns

plt.style.use()
